In [1]:
import pandas as pd
from r import *
from r2 import *
import json

data = pd.read_csv("./dataset.csv")
formule = data['Equation']
print(data)

# for f in range(0, 97):
#     tokens = parse_equation1(formule[f])
#     fix_sorted_tokens = fix2(tokens)
#     sorted_tokens = sort_tokens(fix_sorted_tokens)

#     tree = build_tree(sorted_tokens)
#     rez = calculate_tree(tree, 2)
#     print(f,formule[f],rez )
#     # tree = build_tree(sorted_tokens)
#     # print_infix(tree)


['x', '*', '3', '/', 'x', '+', '-3']
                                  Equation  \
0                           ((x ** 4) - 6)   
1                      (((x / 8) * 2) + 1)   
2                      (((x - 1) - 3) / 5)   
3                                  (x * 5)   
4                                  (x + 2)   
..                                     ...   
93            1*x**5 + -2*x**3 + -1*x + -5   
94              3*x**5 + 2*x**4 + 1*x + -2   
95                     -4*x**5 + -1*x + -5   
96                                4*x + +1   
97   -4*x**5 + 3*x**4 + -4*x**2 + -2*x + 5   

                                                   Xs  \
0   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
1   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
2   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
3   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
4   [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14...   
..                                                ...   
93  [1, 2, 

In [2]:
import random
from r import calculate_tree, fix2
import numpy as np

def mutate_value(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c not in "x*/()-+" and c != "x":
            mutateBool = [True, False, False, False, False]
            if random.choice(mutateBool):
                rand_vals = [str(random.randint(-9, -1)), str(random.randint(1, 9))]
                new_val = random.choice(rand_vals)
                tokens[i] = new_val
    return tokens


def mutate_operation(expression):
    tokens = parse_equation1(expression)
    for i,c in enumerate(tokens):
        # print(c)
        if c in "*/-+^":
            mutateBool = [True, False, False, False, False]
            if random.choice(mutateBool):
                rand_op = ["+", "-", "*","/", "**"]
                new_val = random.choice(rand_op)
                tokens[i] = new_val
    return tokens

def mutate_life(tokens):
    mutation_bool = random.choice([0,0,0,0,0,0,0, 1,1, 2,2, 3])
           
# fix_tokens = fix2(tokens)
# sorted_tokens = sort_tokens(fix_tokens)
# print(sorted_tokens)

# tree1 = build_tree(sorted_tokens)

# tokens2 = mutate_operation(tokens)
# fix_tokens_2 = fix2(tokens2)

# print(fix_tokens_2)

# sorted_tokens_2 = sort_tokens(fix_tokens_2)
# tree2 = build_tree(sorted_tokens_2)


# # print_infix(tree1)
# print(eq)
# print_infix(tree2)


In [6]:
def fitness_primitive(guess_eq, df, index):
    row = df.iloc[index]
    Xs = json.loads(row['Xs'])
    
    Ys = json.loads(row['Ys'])
    real_eq = row['Equation']

    tokens = parse_equation1(guess_eq)

    fix_tokens = fix2(tokens)
    sorted_tokens = sort_tokens(fix_tokens)
    tree = build_tree(sorted_tokens)
    fitnessVal = 0
    for i in range(len(Xs)):
        rez = calculate_tree(tree, int(Xs[i]))
        real_val = int(Ys[i])
        sub = round(np.abs(real_val - rez), 5)
        if sub == 0:
            fitnessVal += 9999
        else:
            fitnessVal += 1/sub
    return fitnessVal
        
    
def generate_expression():
    operators = ["+", "-", "*","/", "**"]
    vals = [str(random.randint(1, 9)), "x"]
    num_terms = random.randint(2, 8)
    terms = []
    x = 0
    for i in range(num_terms):
        op = random.choice(operators)
        if (x == 0 and i == num_terms - 1):
            num = "x"
            x += 1
        else:
            num = random.choice(vals)
            if (num == "x"):
                x += 1
        terms.append(op)
        terms.append(num)

    terms.pop(0)
    expression = "".join(terms)

    return expression

# expression = generate_expression()
# print(expression)
# fitnesVal = fitness_primitive(expression, data, 1)
# print(fitnesVal)

In [8]:
results = {}
# N = 100
def sort_dict(dict):
    return {k: dict[k] for k in sorted(dict, reverse=True)}

def arr_to_string(array):
    return ''.join(map(str, array))



def pick2trees(dictionary):
    e1 = random.choice(tuple(dictionary.items()))
    e2 = random.choice(tuple(dictionary.items()))

    tree_e1 = get_exp_fromArr(str(e1[1]))
    tree_e2 = get_exp_fromArr(str(e2[1]))

    k1 = print_infix(tree_e1)
    k2 = print_infix(tree_e2)

    return tree_e1, tree_e2


    

def ga(N):
    
    for i in range(N):
        expression = generate_expression()
        fitness = fitness_primitive(expression, data, 1)
    
        results[fitness] = expression
    
    N = N // 2
    
    result = sort_dict(results)
    new_dict = dict(list(result.items())[:N])

    while N > 1:
        for ele in new_dict:
            mutation_choice = random.choice([0,0,0,0, 1,1, 2,2,3, 3])

            if mutation_choice == 1:
                expression = arr_to_string(mutate_value(result[ele]))   
                
                fitness = fitness_primitive(expression, data, 1)
                
                result[fitness] = expression
            if mutation_choice == 2:
                
                
                expression = arr_to_string(mutate_operation(result[ele]))
                fitness = fitness_primitive(expression, data, 1)
                result[fitness] = expression

            if mutation_choice == 3:
                tree1, tree2 = pick2trees(new_dict)
                tree1, tree2 = crossover(tree1, tree2)

                #ce ni x v enacbi ponavlja 
                while(check_if_x(tree1) == False) or (check_if_x(tree2) == False):
                    tree1,tree2 = crossover(tree1,tree2)

                tree1_str = infixString(tree1)
                tree2_str = infixString(tree2)
                
                fitness1 = fitness_primitive(tree1_str, data, 1)
                fitness2 = fitness_primitive(tree2_str, data, 1)

                # print(fitness1)
                result[fitness1] = tree1_str
                result[fitness2] = tree2_str

                

        N = N // 2
        print(N)
        result = sort_dict(results)
        new_dict = dict(list(result.items())[:N]) 
        
    return result
            
    
    

k = ga(1000)
val = list(k)[0]

row = data.iloc[1]
Xs = json.loads(row['Xs'])

Ys = json.loads(row['Ys'])
real_eq = row['Equation']
print("---------------------")

print(real_eq)
print("---------------------")

print(k[val], val)

250
125
62
31
15
7
3
1
---------------------
(((x / 8) * 2) + 1)
---------------------
x/5+5 200029.91666666666


In [ ]:
# def fitness_function(ga_instance, Ys,data):
#     expected_Ys = json.loads(data['Ys'])
#     errors = []
#     for i in range(len(Ys)):
#         rez = (expected_Ys[i]-Ys[i])**2
#         errors.append(rez)
#     return sum(errors)/len(errors)

In [ ]:
# MSE = fitness_function(calculate_y,data.iloc[0])
# print(MSE)

In [ ]:
import pygad
ga_instance = pygad.GA(num_generations=100,
                       fitness_func=fitness_function,
                       num_parents_mating=10,
                       sol_per_pop=20,
                       mutation_type="adaptive",
                       mutation_num_genes=(3, 1))

ModuleNotFoundError: No module named 'pygad'